# Appliance repair with Vehicle Routing Problem Solver


In this tutorial, we will use the Python API for VRP to find optimized routes given a set of orders to serve and a set of vehicles with constraints. All you need to do is pass in the "depot" locations, "route" (vehicle) details, the "order" locations, and specify additional properties for orders. The service will return routes with directions. Once you have the results you can add the routes to a map, display the turn-by-turn directions, or integrate them further into your application. To learn more about the capabilities of the routing and directions services, please visit the documentation.

#### Import required libraries and initialise arcgis online portal.

In [1]:
import arcgis
import pandas as pd
import datetime
import getpass
from IPython.display import HTML

from arcgis import geocoding
from arcgis.features import Feature, FeatureSet

In [2]:
portal_url = 'https://www.arcgis.com'
username = getpass.getpass("Enter ArcGIS Online user name:")
password = getpass.getpass("Enter ArcGIS Online user password:")

my_gis = arcgis.GIS(portal_url, username, password)

Enter ArcGIS Online user name:········
Enter ArcGIS Online user password:········



##### Draw map to visualize the inputs and results.


In [32]:
# Create a map of San Fransisco, California.
map_view = my_gis.map('San Fransisco, United States')
#map_view.basemap = 'dark-gray'
map_view

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

### Types of inputs: 
- Geocode or reverse geocode
- csv file
- json file
- online feature service


Read csvs for orders, depots and routes.

In [5]:
orders_csv = "orders.csv"
routes_csv = "routes.csv"
depots_csv = "depots.csv"

#### Routes from csv:
To get a featuresets from dataframe, we convert the CSV to a pandas data frame using read_csv function. Note that in our CSV, `EarliestStartTime` and `LatestStartTime` values are represented as strings denoting time in the local time zone of the computer. So we need to parse these values as date-time values which we accomplish by specifying to_datetime function as the datetime parser.

When calling `arcgis.network.analysis.solve_vehicle_routing_problem` function we need to pass the datetime values in milliseconds since epoch. The `routes_df` dataframe stores these values as datetime type. We convert from datetime to int64 datatype which stores the values in nano seconds. So we convert them to milliseconds.

In [6]:
routes_df = pd.read_csv(routes_csv, parse_dates=["EarliestStartTime", "LatestStartTime"], date_parser=pd.to_datetime)
routes_df["EarliestStartTime"] = routes_df["EarliestStartTime"].astype("int64") / 10 ** 6
routes_df["LatestStartTime"] = routes_df["LatestStartTime"].astype("int64") / 10 ** 6
routes_fs = arcgis.features.FeatureSet.from_dataframe(routes_df)
routes_df

,ObjectID,Name,Description,StartDepotName,EndDepotName,StartDepotServiceTime,EndDepotServiceTime,EarliestStartTime,LatestStartTime,ArriveDepartDelay,...,TotalTravelTime,TotalDistance,StartTime,EndTime,StartTimeUTC,EndTimeUTC,TotalWaitTime,TotalViolationTime,RenewalCount,TotalRenewalServiceTime
0,1,Route1,NaN,Warehouse,Warehouse,NaN,30,-2.209133e+12,-2.209126e+12,NaN,...,91.949035,0,4/4/2018 8:00,4/4/2018 10:05,4/4/2018 8:00,4/4/2018 10:05,0,0,0,0
1,2,Route2,NaN,Warehouse,Warehouse,NaN,30,-2.209133e+12,-2.209126e+12,NaN,...,76.920945,0,4/4/2018 8:00,4/4/2018 9:49,4/4/2018 8:00,4/4/2018 9:49,0,0,0,0


#### Orders from csv :
Orders CSV file has an `Address` column. So we convert orders CSV to a SpatialDataFrame using the address values to geocode order locations.

In [7]:
orders_df = pd.read_csv(orders_csv)
orders_sdf = arcgis.features.SpatialDataFrame.from_df(orders_df, "Address")
orders_fs = orders_sdf.to_featureset()
orders_sdf

,Address,SHAPE
0,"855 La Playa St, San Francisco, California, 94121","{'x': -122.51000717199997, 'y': 37.77222479500..."
1,"90 W Manor Dr, Pacifica, California, 94044","{'x': -122.49172837699996, 'y': 37.64931769600..."
2,"22 Plymouth Ave, Mill Valley, California, 94941","{'x': -122.53238122599998, 'y': 37.89567777300..."
3,"801-899 Curtis St, Albany, California, 94706","{'x': -122.28735777699995, 'y': 37.89263638200..."
4,"3-5 Camino Sobrante, Orinda, California, 94563","{'x': -122.19100216399994, 'y': 37.88477187500..."
5,"San Mateo Superior Court-Northern, 1050 Missio...","{'x': -122.43588998499996, 'y': 37.65866001100..."
6,"T.J. Maxx, 2250 S Shore Ctr, Alameda, CA, 9450...","{'x': -122.25356, 'y': 37.75719000000004}"
7,"6551-6599 Lucas Ave, Oakland, California, 94611","{'x': -122.20783054999998, 'y': 37.82559659900..."
8,"12 Kendell Ct, Sausalito, California, 94965","{'x': -122.50014094799997, 'y': 37.86147050300..."
9,"1595 Shafter Ave, San Francisco, California, 9...","{'x': -122.39053648299995, 'y': 37.73091304700..."


#### Depots from csv:
Depots CSV file has depot locations in columns called `Latitude` (representing Y values) and `Longitude` (representing X values). So we convert depots CSV to a SpatialDataFrame using the X and Y values.

In [9]:
depots_df = pd.read_csv(depots_csv)
depots_sdf = arcgis.features.SpatialDataFrame.from_xy(depots_df, "Longitude", "Latitude")
depots_sdf = depots_sdf.drop(axis=1, labels=["Longitude", "Latitude"])
depots_fs = depots_sdf.to_featureset()
depots_sdf

,Name,SHAPE
0,Warehouse,"{'x': -122.39383000000001, 'spatialReference':..."


### Draw the `depots` and `orders` in map.

In [33]:
map_view_inputs = my_gis.map('San Fransisco, United States', zoomlevel=10)
map_view_inputs

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [34]:
map_view_inputs.draw(orders_fs, symbol={"type": "esriSMS","style": "esriSMSCircle","color": [76,115,0,255],"size": 8})
map_view_inputs.draw(depots_fs, symbol={"type": "esriSMS","style": "esriSMSSquare","color": [255,115,0,255], "size": 10})

### Input : Geocode the `depots` location with the address

In [35]:
depot_geocoded_fs = geocoding.geocode("2-98 Pier 1, San Francisco, California, 94111", as_featureset=True, max_locations=1)
depot_geocoded_fs.df


,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Addr_type,BldgName,BldgType,Block,City,...,UnitName,UnitType,X,Xmax,Xmin,Y,Ymax,Ymin,Zone,SHAPE
0,,98,2,98,2-98,StreetAddress,,,,San Francisco,...,,,-122.39383,-122.39283,-122.39483,37.797505,37.798505,37.796505,,"{'x': -122.39383026489244, 'spatialReference':..."


### Input : JSON file
If you have json files for all the inputs which could be found at the dev lab, you could save the json files on local disk and read those as below. 

In [13]:
import json
with open("Routes_fset.json") as f:
    df1 = json.load(f)
routes_string = json.dumps(df1)    
routes_fset = FeatureSet.from_json(routes_string)
routes_string

'{"features": [{"attributes": {"EarliestStartTime": -2209132800000, "OrderCount": 6, "CostPerUnitTime": 20, "Name": "Laurel", "MaxOrderCount": 6, "MaxTotalTime": 200, "Capacities": "6", "EndDepotName": "Warehouse", "LatestStartTime": -2209125600000, "StartDepotName": "Warehouse", "AssignmentRule": 1, "StartDepotServiceTime": null, "EndDepotServiceTime": 30}}, {"attributes": {"EarliestStartTime": -2209132800000, "OrderCount": 6, "CostPerUnitTime": 20, "Name": "Hardy", "MaxOrderCount": 6, "MaxTotalTime": 200, "Capacities": "6", "EndDepotName": "Warehouse", "LatestStartTime": -2209125600000, "StartDepotName": "Warehouse", "AssignmentRule": 1, "StartDepotServiceTime": null, "EndDepotServiceTime": 30}}]}'

### Input: Online feature service
Connect to the online portal to access feature collection. This is a collection of `orders`, `depots` and `routes`.

In [14]:
# Search for content named as 'VRP_Demo_UC2018_WebLayer'
WebLayersUC_item = my_gis.content.search('title:"VRP_Demo_UC2018_WebLayer" type:Feature Service owner:"shub8703"')

# Search with id
#WebLayersUC = portal.content.get('04e9e0cd84df40a09a03b02b84cbd15d')

WebLayersUC = WebLayersUC_item.pop()
orders_feature_service = WebLayersUC.layers[0].query()
orders_feature_service

<FeatureSet> 10 features

### Solve VRP
You can pass inputs in different formats. For example, depot is geocoded from address, `orders` and `routes` are read from csv files saved on local disk.

In [16]:
today = datetime.datetime.now()
results = arcgis.network.analysis.solve_vehicle_routing_problem(orders= orders_fs,
                                                         depots = depots_fs,
                                                         routes = routes_fs, 
                                                         save_route_data='true',
                                                         populate_directions='true',
                                                         #travel_mode="Driving Time",
                                                         default_date=today)
print('Analysis succeeded? {}'.format(results.solve_succeeded))

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Through Traffic Prohibited").



Analysis succeeded? True


### Result
Let's have a look at the output routes in dataframe.

In [17]:
# Display the output routes in a pandas dataframe.
results.out_routes.df

,DistanceCost,EndTime,EndTimeUTC,Name,ObjectID,OrderCount,OvertimeCost,RegularTimeCost,RenewalCount,Shape_Length,...,TotalCost,TotalDistance,TotalOrderServiceTime,TotalRenewalServiceTime,TotalTime,TotalTravelTime,TotalViolationTime,TotalWaitTime,ViolatedConstraints,SHAPE
0,0,1530873315219,1530898515219,Route1,1,4,0,3105.072862,0,1.020735,...,3105.072862,51.882818,0,0,155.253643,125.253643,0,0,None,"{'paths': [[[-122.3949275, 37.79640750000004],..."
1,0,1530873294780,1530898494780,Route2,2,6,0,3098.259971,0,0.876442,...,3098.259971,55.990276,0,0,154.912999,124.912999,0,0,None,"{'paths': [[[-122.3949275, 37.79640750000004],..."


Now, let us visualize the results on a map.

In [29]:
map_view_outputs = my_gis.map('San Fransisco, United States', zoomlevel=10)
map_view_outputs


MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [36]:
#Visusalize the inputs
map_view_outputs.draw(orders_fs, symbol={"type": "esriSMS","style": "esriSMSCircle","color": [76,115,0,255],"size": 8})
map_view_outputs.draw(depots_fs, symbol={"type": "esriSMS","style": "esriSMSSquare","color": [255,115,0,255], "size": 10})
#Visualize the first route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[0])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, symbol={"type": "esriSLS","style": "esriSLSSolid","color": [0,100,240,255],"size":10})
#Visualize the second route
out_routes_flist = []
out_routes_flist.append(results.out_routes.features[1])
out_routes_fset = []
out_routes_fset = FeatureSet(out_routes_flist)
map_view_outputs.draw(out_routes_fset, symbol={"type": "esriSLS","style": "esriSLSSolid","color": [255,0,0,255],"size":10})

Save the route data from result to local disk which would then be used to upload to onlie portal to share with drivers eventually and share the routes in arcgis online on the portal you have been using. Individual routes are saved as route layers which could then be opened in navigator with directions(if you solve with 'populate_directions'=true')

In [37]:
route_data = results.out_route_data.download()
route_data_item = my_gis.content.add({"type": "File Geodatabase"}, route_data)

Create route layers from the route data. This will create route layers in the online portal which could then be shared with drivers so they would be able to open this navigator.

In [38]:
route_layers = arcgis.features.analysis.create_route_layers(route_data_item, delete_route_data_item=True)
for route_layer in route_layers:
    route_layer.share(org=True)
    display(route_layer.homepage)
    display(route_layer)

'https://transanalytics.maps.arcgis.com/home/item.html?id=9fc31dc91a2f4b58953d33bdf53584b4'

<Item title:"Route2" type:Feature Collection owner:shub8703>

'https://transanalytics.maps.arcgis.com/home/item.html?id=4c9865b292364cc0848e2f472fdbe95e'

<Item title:"Route1" type:Feature Collection owner:shub8703>